In [1]:
from setup import setup_src_path
print(setup_src_path())

['/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/notebooks', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/intial-experments/domain_adaptation_project/modules']


In [2]:
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload

print(config.Config.TXT_SAVE_PATH)
print(config.Config.MODEL_SAVE_PATH)


/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-03-18 11:50:10.515397: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-18 11:50:10.557529: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFl

./text-files/
./hp-model-


In [6]:
reload(config)
reload(processed)
tokenized_data,loaded_data,unsupervised_target = processed.tokenize_and_load_datasets()




Map:   0%|          | 0/75013 [00:00<?, ? examples/s]

Map:   0%|          | 0/8335 [00:00<?, ? examples/s]

In [7]:
source_data = tokenized_data['source']
source_data_eval = tokenized_data['eval_source']
target_data = tokenized_data['target']
test_target_data = tokenized_data['test_target']
test_target_loader = loaded_data['test_target_loader']

In [8]:
unsupervised_target

Dataset({
    features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
    num_rows: 62350
})

In [19]:
from transformers import AutoConfig
from adapters import AutoAdapterModel

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
    num_labels=3,
)
model = AutoAdapterModel.from_pretrained(
    config.Config.MODEL_NAME,
    config=mdlcfg,
)
reload(fn)
fn.print_trainable_parameters(model)

trainable params: 66985530 || all params: 66985530 || trainable%: 100.0


In [20]:
model.delete_adapter("task_adapter")

In [21]:
adapter_name="task_adapter"
model.add_adapter(adapter_name, config="seq_bn")
fn.print_trainable_parameters(model)

trainable params: 67432794 || all params: 67432794 || trainable%: 100.0


In [22]:
model.add_classification_head(
    adapter_name,
    num_labels=3,
  )
model.set_active_adapters(adapter_name)
model.train_adapter(adapter_name)

In [23]:
fn.print_trainable_parameters(model)
model.active_adapters

trainable params: 1662813 || all params: 68025693 || trainable%: 2.444389651421853


Stack[task_adapter]

In [24]:
# Evaluate the model on the target domain before adaptation
reload(fn)
accuracy_before, f1_before = fn.evaluate_model(model, test_target_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.3557692307692308
F1 score before adaptation: 0.3245425280955962


In [25]:
reload(config)
reload(fn)
trainer = fn.train_model(model,"experment_1",source_data,source_data_eval)

/home/guest/.cache/pypoetry/virtualenvs/intial-experments-_CPDD38x-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


  0%|          | 0/9380 [00:00<?, ?it/s]

{'loss': 1.0553, 'learning_rate': 0.0001, 'epoch': 0.21}


  0%|          | 0/31 [00:00<?, ?it/s]

{'eval_loss': 0.9037796854972839, 'eval_accuracy': 0.576417004048583, 'eval_f1': 0.5780329102533986, 'eval_precision': 0.5866149796608384, 'eval_recall': 0.576417004048583, 'eval_runtime': 2.5655, 'eval_samples_per_second': 770.229, 'eval_steps_per_second': 12.084, 'epoch': 0.21}
{'loss': 0.8667, 'learning_rate': 9.436936936936938e-05, 'epoch': 0.43}


...

{'eval_loss': 0.7097656726837158, 'eval_accuracy': 0.7090080971659919, 'eval_f1': 0.7095148656707215, 'eval_precision': 0.72572039610032, 'eval_recall': 0.7090080971659919, 'eval_runtime': 2.6364, 'eval_samples_per_second': 749.508, 'eval_steps_per_second': 11.758, 'epoch': 3.62}
{'loss': 0.6427, 'learning_rate': 4.279279279279279e-06, 'epoch': 3.84}


  0%|          | 0/31 [00:00<?, ?it/s]

{'eval_loss': 0.7217413187026978, 'eval_accuracy': 0.701417004048583, 'eval_f1': 0.7023359045444422, 'eval_precision': 0.7268274205759007, 'eval_recall': 0.701417004048583, 'eval_runtime': 2.6227, 'eval_samples_per_second': 753.427, 'eval_steps_per_second': 11.82, 'epoch': 3.84}


Overwriting existing adapter 'task_adapter'.
Overwriting existing head 'task_adapter'


{'train_runtime': 842.5509, 'train_samples_per_second': 356.123, 'train_steps_per_second': 11.133, 'train_loss': 0.7268655284889726, 'epoch': 4.0}


In [26]:
trainer.evaluate(test_target_data)

  0%|          | 0/31 [00:00<?, ?it/s]

{'eval_loss': 0.7097656726837158,
 'eval_accuracy': 0.7090080971659919,
 'eval_f1': 0.7095148656707215,
 'eval_precision': 0.72572039610032,
 'eval_recall': 0.7090080971659919,
 'eval_runtime': 2.4934,
 'eval_samples_per_second': 792.5,
 'eval_steps_per_second': 12.433,
 'epoch': 4.0}

In [29]:
accuracy_before, f1_before = fn.evaluate_model(trainer.model, test_target_loader)
print(f"Accuracy before adaptation: {accuracy_before}")
print(f"F1 score before adaptation: {f1_before}")

Accuracy before adaptation: 0.7090080971659919
F1 score before adaptation: 0.7095148656707215


In [28]:
reload(config)
source_data.save_to_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data")
source_data_eval.save_to_disk(f"{config.Config.DATASETS_SAVE_PATH}/source_data_eval")
target_data.save_to_disk(f"{config.Config.DATASETS_SAVE_PATH}/target_data")
test_target_data.save_to_disk(f"{config.Config.DATASETS_SAVE_PATH}/test_target_data")
unsupervised_target.save_to_disk(f"{config.Config.DATASETS_SAVE_PATH}/unsupervised_target")


Saving the dataset (0/1 shards):   0%|          | 0/75013 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1976 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/15000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1976 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/62350 [00:00<?, ? examples/s]

In [31]:
trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name)

In [32]:
from datasets import load_from_disk

reloaded_unsupervised_target = load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/unsupervised_target")
# Similarly for the other datasets
reloaded_unsupervised_target

Dataset({
    features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label'],
    num_rows: 62350
})